# Selenium / Playwright 库学习指南

## 目录
1. [Selenium/Playwright简介](#seleniumplaywright简介)
2. [为什么选择Selenium/Playwright](#为什么选择seleniumplaywright)
3. [安装与环境配置](#安装与环境配置)
4. [基础操作：打开网页](#基础操作打开网页)
5. [元素定位与交互](#元素定位与交互)
6. [等待机制](#等待机制)
7. [表单填写与提交](#表单填写与提交)
8. [截图与页面信息获取](#截图与页面信息获取)
9. [HTTP协议基础](#http协议基础)
10. [网络安全与合规性](#网络安全与合规性)
11. [实践练习](#实践练习)


## Selenium/Playwright简介

**Selenium** 和 **Playwright** 是两个强大的浏览器自动化工具，可以模拟真实用户操作浏览器，实现网页自动化测试、数据采集等功能。

### Selenium
- 历史悠久的浏览器自动化框架
- 支持多种浏览器（Chrome、Firefox、Safari等）
- 广泛使用的WebDriver协议
- 社区庞大，资源丰富

### Playwright
- 微软开发的现代化浏览器自动化工具
- 支持多浏览器（Chromium、Firefox、WebKit）
- 更快的执行速度和更稳定的API
- 内置等待机制，减少时序问题
- 支持网络拦截和模拟

### 主要应用场景
- **Web自动化测试**：功能测试、回归测试
- **数据采集**：处理JavaScript渲染的动态网页
- **UI自动化**：重复性操作自动化
- **性能监控**：页面加载时间分析


## 为什么选择Selenium/Playwright

### 相比其他爬虫库的优势

#### 1. **处理JavaScript渲染**
- **requests/urllib**：只能获取静态HTML，无法执行JavaScript
- **Selenium/Playwright**：完整浏览器环境，可以执行所有JavaScript代码
- **应用场景**：现代单页应用（SPA）、动态加载内容

#### 2. **真实浏览器环境**
- **requests库**：只发送HTTP请求，容易被反爬虫机制识别
- **Selenium/Playwright**：使用真实浏览器，行为更像人类用户
- **优势**：可以处理Cookie、Session、JavaScript验证等

#### 3. **交互式操作**
- **其他库**：只能发送请求，无法点击、滚动、填写表单
- **Selenium/Playwright**：可以模拟鼠标、键盘操作
- **应用**：需要登录、填写表单、点击按钮的场景

#### 4. **网络拦截与模拟**
- **Playwright特有**：可以拦截和修改网络请求
- **应用**：API测试、性能优化、请求分析

#### 5. **多浏览器支持**
- 可以在不同浏览器中测试，确保兼容性
- 支持无头模式（headless），节省资源


## 安装与环境配置

### 安装Selenium
```bash
pip install selenium
```

### 安装Playwright
```bash
pip install playwright
playwright install  # 安装浏览器驱动
```

### 浏览器驱动说明
- **Selenium**：需要单独下载浏览器驱动（ChromeDriver、GeckoDriver等）
- **Playwright**：自动管理浏览器驱动，更便捷


In [ ]:
# 检查已安装的库版本
try:
    import selenium
    print(f"Selenium版本: {selenium.__version__}")
except ImportError:
    print("Selenium未安装，请运行: pip install selenium")

try:
    import playwright
    print(f"Playwright版本: {playwright.__version__}")
except ImportError:
    print("Playwright未安装，请运行: pip install playwright")


## 基础操作：打开网页

### HTTP请求流程说明

当我们在浏览器中输入网址时，发生了什么？

1. **DNS解析**：将域名（如 www.example.com）转换为IP地址
   - 就像查电话簿，找到域名对应的服务器地址
   
2. **TCP连接**：与服务器建立TCP连接（三次握手）
   - 客户端："我想连接" → 服务器："好的，可以连接" → 客户端："收到，开始传输"
   
3. **HTTP请求**：发送HTTP请求报文
   ```
   GET /index.html HTTP/1.1
   Host: www.example.com
   User-Agent: Mozilla/5.0...
   ```
   
4. **服务器响应**：服务器返回HTML、CSS、JavaScript等资源
   
5. **浏览器渲染**：浏览器解析HTML，执行JavaScript，显示页面

**Selenium/Playwright的优势**：它们控制真实浏览器，自动完成上述所有步骤！


In [ ]:
# ========== Selenium 基础示例：打开网页 ==========

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import time

# 配置Chrome选项
chrome_options = Options()
# headless模式：无界面运行，节省资源
# 注释掉下面这行可以看到浏览器窗口
# chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')  # 在Linux系统上可能需要
chrome_options.add_argument('--disable-dev-shm-usage')  # 解决资源限制问题

# 创建浏览器驱动
# 注意：需要下载ChromeDriver并放在PATH中，或指定路径
# driver = webdriver.Chrome(service=Service('path/to/chromedriver'), options=chrome_options)
driver = webdriver.Chrome(options=chrome_options)

try:
    # 打开网页
    # 这里使用一个测试网站，实际使用时请遵守robots.txt和网站服务条款
    url = "https://www.example.com"
    driver.get(url)
    
    # 获取页面标题
    print(f"页面标题: {driver.title}")
    
    # 获取当前URL
    print(f"当前URL: {driver.current_url}")
    
    # 等待几秒以便观察（实际使用中不需要）
    time.sleep(2)
    
finally:
    # 重要：关闭浏览器，释放资源
    driver.quit()


In [ ]:
# ========== Playwright 基础示例：打开网页 ==========

from playwright.sync_api import sync_playwright
import time

# Playwright使用上下文管理器，更安全
with sync_playwright() as p:
    # 启动浏览器（chromium、firefox或webkit）
    browser = p.chromium.launch(
        headless=False  # False表示显示浏览器窗口，True表示无头模式
    )
    
    # 创建浏览器上下文（类似打开一个新的浏览器会话）
    # 上下文可以设置Cookie、用户代理等
    context = browser.new_context(
        viewport={'width': 1920, 'height': 1080},  # 设置视口大小
        user_agent='Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'  # 设置用户代理
    )
    
    # 创建新页面（类似打开新标签页）
    page = context.new_page()
    
    try:
        # 访问网页
        # 注意：请遵守网站的robots.txt和使用条款
        page.goto("https://www.example.com", wait_until='networkidle')  # 等待网络空闲
        
        # 获取页面标题
        print(f"页面标题: {page.title()}")
        
        # 获取当前URL
        print(f"当前URL: {page.url}")
        
        time.sleep(2)
        
    finally:
        # 关闭浏览器
        browser.close()


## 元素定位与交互

### DOM（文档对象模型）基础

**DOM** 是浏览器将HTML文档解析成的树状结构，每个HTML标签都是一个节点（元素）。

```
HTML结构：
<html>
  <body>
    <div id="main">
      <h1>标题</h1>
      <button class="btn">点击</button>
    </div>
  </body>
</html>

DOM树：
html
 └── body
      └── div#main
           ├── h1 ("标题")
           └── button.btn ("点击")
```

**元素定位**：通过ID、类名、标签名、XPath、CSS选择器等方式找到页面上的元素。

### 定位方式对比

| 方式 | 示例 | 特点 |
|------|------|------|
| ID | `id="username"` | 唯一标识，最快 |
| 类名 | `class="btn"` | 可能多个元素 |
| 标签名 | `<button>` | 可能有多个 |
| CSS选择器 | `#id .class` | 灵活强大 |
| XPath | `/html/body/div` | 功能强大但复杂 |


In [ ]:
# ========== Selenium 元素定位示例 ==========

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time

chrome_options = Options()
# chrome_options.add_argument('--headless')
driver = webdriver.Chrome(options=chrome_options)

try:
    # 访问一个包含表单的测试页面
    # 实际使用中，请使用合法的测试网站或本地HTML文件
    driver.get("https://www.example.com")
    
    # 方式1：通过ID定位（最常用，最快速）
    # 假设页面有 <input id="search"> 元素
    # search_input = driver.find_element(By.ID, "search")
    
    # 方式2：通过类名定位
    # 假设页面有 <button class="submit-btn"> 元素
    # submit_btn = driver.find_element(By.CLASS_NAME, "submit-btn")
    
    # 方式3：通过CSS选择器定位（推荐，灵活）
    # 假设页面有 <div class="container"><a href="/link">链接</a></div>
    # link = driver.find_element(By.CSS_SELECTOR, "div.container a")
    
    # 方式4：通过XPath定位（功能强大但较慢）
    # link = driver.find_element(By.XPATH, "//div[@class='container']//a")
    
    # 方式5：通过标签名定位
    # 获取所有链接
    # links = driver.find_elements(By.TAG_NAME, "a")  # 注意：find_elements返回列表
    
    # 示例：查找并点击链接
    # 注意：以下代码需要根据实际页面元素调整
    # link = driver.find_element(By.PARTIAL_LINK_TEXT, "More")  # 通过链接文本的部分匹配
    # link.click()  # 点击元素
    
    print("元素定位示例完成")
    time.sleep(2)
    
except Exception as e:
    print(f"错误: {e}")
finally:
    driver.quit()


In [ ]:
# ========== Playwright 元素定位示例 ==========

from playwright.sync_api import sync_playwright
import time

with sync_playwright() as p:
    browser = p.chromium.launch(headless=False)
    page = browser.new_page()
    
    try:
        page.goto("https://www.example.com", wait_until='networkidle')
        
        # Playwright的定位方式（更简洁）
        # 方式1：通过文本内容定位
        # page.click("text=点击我")  # 点击包含"点击我"文本的元素
        
        # 方式2：通过CSS选择器
        # page.click("button.submit-btn")  # 点击class为submit-btn的button
        
        # 方式3：通过ID
        # page.fill("#username", "testuser")  # 在id为username的输入框填写内容
        
        # 方式4：通过属性
        # page.click("[data-testid='submit']")  # 点击data-testid属性为submit的元素
        
        # 方式5：组合定位
        # page.click("div.container >> button")  # 在container div内的button
        
        # 获取元素文本内容
        # title = page.locator("h1").text_content()
        # print(f"标题: {title}")
        
        # 获取所有匹配的元素
        # links = page.locator("a").all()  # 获取所有链接
        # for link in links:
        #     print(link.text_content())
        
        print("元素定位示例完成")
        time.sleep(2)
        
    finally:
        browser.close()


## 等待机制

### 为什么需要等待？

网页是**异步加载**的：
1. HTML先加载完成
2. CSS样式文件异步加载
3. JavaScript文件异步加载和执行
4. 图片、视频等资源异步加载
5. AJAX请求异步获取数据

**问题**：如果代码在元素加载完成前就尝试操作，会报错！

**解决方案**：使用等待机制，确保元素存在后再操作。

### 等待类型

1. **隐式等待**：设置全局等待时间，所有操作都会等待
2. **显式等待**：等待特定条件满足（推荐）
3. **固定等待**：`time.sleep()`，不推荐（效率低）


In [ ]:
# ========== Selenium 等待机制示例 ==========

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
import time

chrome_options = Options()
driver = webdriver.Chrome(options=chrome_options)

try:
    # 方式1：隐式等待（不推荐，效率低）
    # 设置全局等待时间，所有find_element操作最多等待10秒
    driver.implicitly_wait(10)  # 单位：秒
    
    driver.get("https://www.example.com")
    
    # 方式2：显式等待（推荐）
    # 等待特定元素出现，最多等待10秒
    # 如果10秒内元素出现，立即继续；如果10秒后仍未出现，抛出异常
    wait = WebDriverWait(driver, 10)  # 创建等待对象，最多等待10秒
    
    # 等待元素可见（不仅存在，还要可见）
    # element = wait.until(EC.visibility_of_element_located((By.ID, "target-element")))
    
    # 等待元素可点击
    # element = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button.submit")))
    
    # 等待页面标题包含特定文本
    # wait.until(EC.title_contains("Example"))
    
    # 等待URL变化
    # wait.until(EC.url_contains("success"))
    
    # 方式3：固定等待（不推荐，除非必要）
    # time.sleep(5)  # 无论页面是否加载完成，都等待5秒
    
    print("等待机制示例完成")
    
except Exception as e:
    print(f"错误: {e}")
finally:
    driver.quit()


In [ ]:
# ========== Playwright 等待机制示例 ==========

from playwright.sync_api import sync_playwright

with sync_playwright() as p:
    browser = p.chromium.launch(headless=False)
    page = browser.new_page()
    
    try:
        # Playwright的等待机制更智能，默认自动等待
        
        # 方式1：goto时等待（推荐）
        # networkidle: 等待网络空闲（500ms内无网络请求）
        # load: 等待load事件
        # domcontentloaded: 等待DOM内容加载完成
        page.goto("https://www.example.com", wait_until='networkidle')
        
        # 方式2：操作时自动等待
        # Playwright的所有操作（click、fill等）都会自动等待元素可见和可操作
        # page.click("button")  # 自动等待button可见且可点击
        
        # 方式3：显式等待元素
        # 等待元素出现
        # page.wait_for_selector("div.content", state="visible")  # 等待div.content可见
        
        # 等待元素消失
        # page.wait_for_selector("div.loading", state="hidden")  # 等待loading消失
        
        # 方式4：等待特定条件
        # 等待函数返回True
        # page.wait_for_function("document.querySelector('div').textContent.includes('完成')")
        
        # 方式5：等待网络请求
        # 等待特定API请求完成
        # with page.expect_response("https://api.example.com/data") as response_info:
        #     page.click("button.load-data")
        # response = response_info.value
        # print(f"响应状态: {response.status}")
        
        print("等待机制示例完成")
        
    finally:
        browser.close()


## 表单填写与提交

### HTTP POST请求说明

当提交表单时，浏览器会发送**POST请求**：

```
POST /login HTTP/1.1
Host: www.example.com
Content-Type: application/x-www-form-urlencoded
Content-Length: 25

username=test&password=123
```

**关键点**：
- **GET请求**：参数在URL中（`?key=value`），用于获取数据
- **POST请求**：参数在请求体中，用于提交数据（更安全）
- **Content-Type**：告诉服务器数据的格式

**Selenium/Playwright的优势**：自动处理表单序列化、编码等复杂操作！


In [ ]:
# ========== Selenium 表单填写示例 ==========

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
import time

chrome_options = Options()
driver = webdriver.Chrome(options=chrome_options)

try:
    # 注意：以下示例使用假设的表单，实际使用时请遵守网站服务条款
    # 合规提醒：不要使用自动化工具绕过网站的安全验证（如验证码）
    # 不要进行未授权的登录尝试，这可能违反计算机安全法规
    
    driver.get("https://www.example.com")
    
    # 等待表单元素加载
    wait = WebDriverWait(driver, 10)
    
    # 示例1：填写文本输入框
    # username_input = wait.until(EC.presence_of_element_located((By.ID, "username")))
    # username_input.clear()  # 清空已有内容
    # username_input.send_keys("testuser")  # 输入文本
    
    # 示例2：填写密码框
    # password_input = driver.find_element(By.ID, "password")
    # password_input.send_keys("password123")
    
    # 示例3：选择下拉框
    # from selenium.webdriver.support.ui import Select
    # select = Select(driver.find_element(By.ID, "country"))
    # select.select_by_visible_text("中国")  # 通过可见文本选择
    # # 或 select.select_by_value("CN")  # 通过value值选择
    # # 或 select.select_by_index(0)  # 通过索引选择
    
    # 示例4：选择复选框或单选框
    # checkbox = driver.find_element(By.ID, "agree")
    # if not checkbox.is_selected():  # 检查是否已选中
    #     checkbox.click()
    
    # 示例5：使用键盘操作
    # input_field = driver.find_element(By.ID, "search")
    # input_field.send_keys("搜索内容")
    # input_field.send_keys(Keys.RETURN)  # 按回车键
    
    # 示例6：提交表单
    # 方式1：点击提交按钮
    # submit_btn = driver.find_element(By.CSS_SELECTOR, "button[type='submit']")
    # submit_btn.click()
    
    # 方式2：在输入框中按回车
    # password_input.send_keys(Keys.RETURN)
    
    # 方式3：调用表单的submit方法
    # form = driver.find_element(By.ID, "login-form")
    # form.submit()
    
    print("表单填写示例完成")
    time.sleep(2)
    
except Exception as e:
    print(f"错误: {e}")
finally:
    driver.quit()


In [ ]:
# ========== Playwright 表单填写示例 ==========

from playwright.sync_api import sync_playwright
import time

with sync_playwright() as p:
    browser = p.chromium.launch(headless=False)
    page = browser.new_page()
    
    try:
        page.goto("https://www.example.com", wait_until='networkidle')
        
        # 合规提醒：
        # 1. 仅用于授权的测试环境
        # 2. 不要绕过验证码等安全机制
        # 3. 遵守网站的robots.txt和使用条款
        # 4. 不要进行暴力破解等非法操作
        
        # 示例1：填写输入框（自动等待元素可见）
        # page.fill("#username", "testuser")  # fill会先清空再填写
        
        # 示例2：逐字符输入（模拟真实打字）
        # page.type("#username", "testuser", delay=100)  # delay是每个字符之间的延迟（毫秒）
        
        # 示例3：追加文本（不清空）
        # page.fill("#username", "test")
        # page.press("#username", "End")  # 光标移到末尾
        # page.type("#username", "user")  # 追加文本
        
        # 示例4：选择下拉框
        # page.select_option("#country", "CN")  # 通过value选择
        # # 或 page.select_option("#country", label="中国")  # 通过标签选择
        
        # 示例5：选择复选框/单选框
        # page.check("#agree")  # 选中
        # page.uncheck("#agree")  # 取消选中
        
        # 示例6：点击按钮
        # page.click("button[type='submit']")
        
        # 示例7：键盘操作
        # page.fill("#search", "关键词")
        # page.press("#search", "Enter")  # 按回车键
        # # 其他按键：Tab, Shift+Tab, ArrowDown, ArrowUp等
        
        # 示例8：文件上传
        # page.set_input_files("#file-upload", "path/to/file.txt")
        
        # 示例9：处理JavaScript弹窗（alert、confirm、prompt）
        # page.on("dialog", lambda dialog: dialog.accept())  # 自动接受弹窗
        # page.click("button.trigger-alert")
        
        print("表单填写示例完成")
        time.sleep(2)
        
    finally:
        browser.close()


## 截图与页面信息获取

### 为什么需要截图？

1. **调试**：查看页面实际渲染效果
2. **测试报告**：记录测试过程中的页面状态
3. **数据验证**：确认获取的数据是否正确
4. **错误记录**：保存错误发生时的页面快照


In [ ]:
# ========== Selenium 截图与信息获取示例 ==========

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time

chrome_options = Options()
driver = webdriver.Chrome(options=chrome_options)

try:
    driver.get("https://www.example.com")
    time.sleep(2)
    
    # 截图功能
    # 保存整个页面截图
    driver.save_screenshot("screenshot.png")  # 保存为PNG格式
    
    # 获取页面信息
    print(f"页面标题: {driver.title}")
    print(f"当前URL: {driver.current_url}")
    print(f"页面源代码长度: {len(driver.page_source)} 字符")
    
    # 获取窗口大小
    size = driver.get_window_size()
    print(f"窗口大小: {size['width']} x {size['height']}")
    
    # 执行JavaScript获取更多信息
    # 获取页面高度（包括滚动部分）
    page_height = driver.execute_script("return document.body.scrollHeight")
    print(f"页面高度: {page_height} 像素")
    
    # 获取所有Cookie
    cookies = driver.get_cookies()
    print(f"Cookie数量: {len(cookies)}")
    for cookie in cookies:
        print(f"  - {cookie['name']}: {cookie['value']}")
    
    # 获取页面所有链接
    links = driver.find_elements(By.TAG_NAME, "a")
    print(f"链接数量: {len(links)}")
    for link in links[:5]:  # 只显示前5个
        print(f"  - {link.get_attribute('href')}")
    
except Exception as e:
    print(f"错误: {e}")
finally:
    driver.quit()


In [ ]:
# ========== Playwright 截图与信息获取示例 ==========

from playwright.sync_api import sync_playwright
import time

with sync_playwright() as p:
    browser = p.chromium.launch(headless=False)
    page = browser.new_page()
    
    try:
        page.goto("https://www.example.com", wait_until='networkidle')
        time.sleep(2)
        
        # 截图功能（Playwright更强大）
        # 全页面截图
        page.screenshot(path="screenshot_full.png", full_page=True)  # full_page=True包含滚动部分
        
        # 元素截图
        # element = page.locator("div.content")
        # element.screenshot(path="element_screenshot.png")
        
        # 获取页面信息
        print(f"页面标题: {page.title()}")
        print(f"当前URL: {page.url}")
        
        # 获取页面内容
        content = page.content()  # 获取HTML源代码
        print(f"页面源代码长度: {len(content)} 字符")
        
        # 执行JavaScript
        page_height = page.evaluate("document.body.scrollHeight")
        print(f"页面高度: {page_height} 像素")
        
        # 获取Cookie
        cookies = page.context.cookies()
        print(f"Cookie数量: {len(cookies)}")
        for cookie in cookies:
            print(f"  - {cookie['name']}: {cookie['value']}")
        
        # 获取所有链接
        links = page.locator("a").all()
        print(f"链接数量: {len(links)}")
        for link in links[:5]:
            href = link.get_attribute("href")
            text = link.text_content()
            print(f"  - {text}: {href}")
        
        # 获取网络请求信息（Playwright特有）
        # 监听所有网络请求
        # def handle_request(request):
        #     print(f"请求: {request.method} {request.url}")
        # def handle_response(response):
        #     print(f"响应: {response.status} {response.url}")
        # page.on("request", handle_request)
        # page.on("response", handle_response)
        
    finally:
        browser.close()


## HTTP协议基础

### HTTP请求与响应结构

#### HTTP请求（Request）

```
GET /index.html HTTP/1.1          ← 请求行（方法、路径、协议版本）
Host: www.example.com              ← 请求头（Header）
User-Agent: Mozilla/5.0...
Accept: text/html
Cookie: session_id=abc123
                                    ← 空行（分隔头部和主体）
                                    ← 请求体（Body，GET请求通常为空）
```

**请求方法**：
- **GET**：获取资源（如打开网页）
- **POST**：提交数据（如登录、提交表单）
- **PUT**：更新资源
- **DELETE**：删除资源

**请求头（Header）**：
- **User-Agent**：告诉服务器使用的浏览器类型
- **Cookie**：存储会话信息（如登录状态）
- **Referer**：来源页面URL
- **Accept**：客户端能接受的响应类型

#### HTTP响应（Response）

```
HTTP/1.1 200 OK                    ← 状态行（协议版本、状态码、状态文本）
Content-Type: text/html            ← 响应头
Content-Length: 1234
Set-Cookie: session_id=abc123
                                    ← 空行
<!DOCTYPE html>                    ← 响应体（Body）
<html>...
```

**状态码**：
- **200 OK**：请求成功
- **301/302**：重定向
- **404 Not Found**：资源不存在
- **500 Internal Server Error**：服务器错误

### HTTPS与安全

**HTTPS = HTTP + SSL/TLS加密**

```
HTTP请求  →  [加密]  →  HTTPS请求  →  服务器
明文传输       SSL/TLS      加密传输
```

**SSL/TLS作用**：
1. **加密**：防止数据被窃听
2. **身份验证**：确认服务器身份（防止中间人攻击）
3. **完整性**：确保数据未被篡改

**证书验证**：
- 浏览器会验证服务器的SSL证书
- 证书由CA（证书颁发机构）签发
- 如果证书无效，浏览器会显示警告


In [ ]:
# ========== 查看HTTP请求和响应示例 ==========

from playwright.sync_api import sync_playwright

with sync_playwright() as p:
    browser = p.chromium.launch(headless=False)
    page = browser.new_page()
    
    try:
        # 监听所有网络请求
        def log_request(request):
            print(f"\n=== HTTP请求 ===")
            print(f"方法: {request.method}")  # GET, POST等
            print(f"URL: {request.url}")
            print(f"请求头:")
            for name, value in request.headers.items():
                print(f"  {name}: {value}")
            if request.post_data:
                print(f"请求体: {request.post_data}")
        
        def log_response(response):
            print(f"\n=== HTTP响应 ===")
            print(f"URL: {response.url}")
            print(f"状态码: {response.status}")  # 200, 404等
            print(f"状态文本: {response.status_text}")  # OK, Not Found等
            print(f"响应头:")
            for name, value in response.headers.items():
                print(f"  {name}: {value}")
            # 注意：获取响应体需要异步操作，这里只是示例
        
        page.on("request", log_request)
        page.on("response", log_response)
        
        # 访问网页，触发网络请求
        page.goto("https://www.example.com", wait_until='networkidle')
        
        print("\n网络请求监听完成")
        
    finally:
        browser.close()


## 网络安全与合规性

### 常见网络安全威胁

#### 1. **中间人攻击（Man-in-the-Middle）**

```
客户端  →  [攻击者]  →  服务器
        拦截和篡改数据
```

**防护**：使用HTTPS，验证SSL证书

#### 2. **XSS（跨站脚本攻击）**

恶意脚本注入到网页中，窃取用户信息。

**防护**：输入验证、输出转义

#### 3. **CSRF（跨站请求伪造）**

利用用户已登录的状态，伪造请求。

**防护**：CSRF Token、SameSite Cookie

#### 4. **SQL注入**

在输入中注入SQL代码，攻击数据库。

**防护**：参数化查询、输入验证

### 合规性说明

#### ⚠️ 重要法律和道德提醒

1. **遵守robots.txt**
   - 网站通过robots.txt声明哪些页面可以爬取
   - 违反robots.txt可能违反网站服务条款

2. **遵守网站服务条款**
   - 大多数网站禁止自动化访问
   - 违反可能导致法律后果

3. **不要绕过安全机制**
   - 不要绕过验证码（可能违法）
   - 不要进行暴力破解
   - 不要绕过登录限制

4. **尊重服务器资源**
   - 添加适当的延迟（避免频繁请求）
   - 不要对服务器造成过大负担
   - 使用缓存减少重复请求

5. **数据使用合规**
   - 遵守数据保护法规（如GDPR）
   - 不要收集个人敏感信息
   - 遵守版权法律

6. **仅用于合法目的**
   - 测试自己的网站
   - 授权的数据采集
   - 学术研究（需获得许可）

#### 最佳实践

```python
# 示例：添加延迟，避免过于频繁的请求
import time
import random

def respectful_request(page, url):
    """尊重服务器的请求函数"""
    # 随机延迟1-3秒，模拟人类行为
    time.sleep(random.uniform(1, 3))
    page.goto(url)
    
# 检查robots.txt（可以使用urllib.robotparser）
from urllib.robotparser import RobotFileParser

rp = RobotFileParser()
rp.set_url("https://www.example.com/robots.txt")
rp.read()
if rp.can_fetch("MyBot", "https://www.example.com/page"):
    # 允许访问
    pass
else:
    # 不允许访问
    pass
```


In [ ]:
# ========== 合规的爬虫实践示例 ==========

from playwright.sync_api import sync_playwright
import time
import random
from urllib.robotparser import RobotFileParser

def check_robots_txt(base_url, user_agent="MyBot/1.0"):
    """
    检查robots.txt，判断是否允许访问
    
    参数:
        base_url: 网站基础URL（如 "https://www.example.com"）
        user_agent: 爬虫的用户代理名称
    
    返回:
        bool: 是否允许访问
    """
    try:
        rp = RobotFileParser()
        robots_url = f"{base_url}/robots.txt"
        rp.set_url(robots_url)
        rp.read()
        
        # 检查是否允许访问根路径
        can_fetch = rp.can_fetch(user_agent, base_url)
        print(f"robots.txt检查: {'允许' if can_fetch else '禁止'}访问 {base_url}")
        return can_fetch
    except Exception as e:
        print(f"无法读取robots.txt: {e}")
        # 如果无法读取，保守处理：不允许访问
        return False

def respectful_crawl(page, url, min_delay=1, max_delay=3):
    """
    尊重服务器的爬取函数
    
    参数:
        page: Playwright页面对象
        url: 要访问的URL
        min_delay: 最小延迟（秒）
        max_delay: 最大延迟（秒）
    """
    # 添加随机延迟，模拟人类行为
    delay = random.uniform(min_delay, max_delay)
    print(f"等待 {delay:.2f} 秒后访问 {url}")
    time.sleep(delay)
    
    # 访问页面
    try:
        page.goto(url, wait_until='networkidle', timeout=30000)
        return True
    except Exception as e:
        print(f"访问失败: {e}")
        return False

# 示例使用
with sync_playwright() as p:
    browser = p.chromium.launch(headless=True)
    page = browser.new_page()
    
    # 设置合理的User-Agent
    page.set_extra_http_headers({
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36"
    })
    
    try:
        base_url = "https://www.example.com"
        
        # 1. 检查robots.txt
        if not check_robots_txt(base_url):
            print("根据robots.txt，不允许访问此网站")
            # 在实际应用中，应该停止爬取
            # return
        
        # 2. 合规地访问页面
        if respectful_crawl(page, base_url):
            print("页面访问成功")
            # 进行数据提取等操作
        
    finally:
        browser.close()

print("\n合规爬虫示例完成")
print("\n提醒：")
print("1. 始终遵守robots.txt")
print("2. 遵守网站服务条款")
print("3. 添加适当的延迟")
print("4. 不要绕过安全机制")
print("5. 仅用于合法目的")


In [ ]:
# ========== 实践练习：数据采集并创建表格 ==========

from playwright.sync_api import sync_playwright
import pandas as pd  # 用于创建表格
import time

# 创建一个示例：采集网页信息并整理成表格
def collect_page_info():
    """
    采集网页信息并整理成表格
    
    注意：这是一个示例，实际使用时请遵守网站服务条款
    """
    data = []  # 存储采集的数据
    
    with sync_playwright() as p:
        browser = p.chromium.launch(headless=False)
        page = browser.new_page()
        
        try:
            # 访问网页
            page.goto("https://www.example.com", wait_until='networkidle')
            time.sleep(2)
            
            # 采集页面基本信息
            page_info = {
                '页面标题': page.title(),
                'URL': page.url,
                '页面高度': page.evaluate("document.body.scrollHeight"),
                '链接数量': len(page.locator("a").all()),
                '图片数量': len(page.locator("img").all()),
            }
            data.append(page_info)
            
            # 采集所有链接信息
            links = page.locator("a").all()
            for i, link in enumerate(links[:10]):  # 只采集前10个链接作为示例
                try:
                    href = link.get_attribute("href") or ""
                    text = link.text_content() or ""
                    link_info = {
                        '链接序号': i + 1,
                        '链接文本': text[:50],  # 限制文本长度
                        '链接地址': href[:100],  # 限制URL长度
                        '是否外部链接': href.startswith('http') and 'example.com' not in href
                    }
                    data.append(link_info)
                except Exception as e:
                    print(f"采集链接 {i+1} 时出错: {e}")
            
        finally:
            browser.close()
    
    return data

# 执行采集
print("开始采集数据...")
collected_data = collect_page_info()

# 创建DataFrame表格
if collected_data:
    df = pd.DataFrame(collected_data)
    
    # 显示表格
    print("\n采集的数据表格：")
    print(df.to_string(index=False))
    
    # 保存为CSV文件
    df.to_csv("采集数据.csv", index=False, encoding='utf-8-sig')
    print("\n数据已保存到 '采集数据.csv'")
    
    # 显示统计信息
    print(f"\n统计信息：")
    print(f"总记录数: {len(df)}")
    if '链接地址' in df.columns:
        external_links = df[df.get('是否外部链接', False) == True]
        print(f"外部链接数: {len(external_links)}")
else:
    print("未采集到数据")


In [ ]:
# ========== 进阶示例：多页面数据采集表格 ==========

from playwright.sync_api import sync_playwright
import pandas as pd
import time
import random

def collect_multiple_pages(urls):
    """
    采集多个页面的信息并整理成表格
    
    参数:
        urls: 要采集的URL列表
    
    返回:
        DataFrame: 包含所有页面信息的表格
    """
    all_data = []
    
    with sync_playwright() as p:
        browser = p.chromium.launch(headless=True)  # 无头模式，更快
        context = browser.new_context(
            viewport={'width': 1920, 'height': 1080}
        )
        page = context.new_page()
        
        for i, url in enumerate(urls, 1):
            try:
                print(f"正在采集第 {i}/{len(urls)} 个页面: {url}")
                
                # 添加延迟，避免过于频繁的请求
                if i > 1:
                    delay = random.uniform(2, 4)
                    time.sleep(delay)
                
                # 访问页面
                page.goto(url, wait_until='networkidle', timeout=30000)
                
                # 采集信息
                page_data = {
                    '序号': i,
                    '页面标题': page.title(),
                    'URL': page.url,
                    '状态码': '200',  # 实际可以通过网络监听获取
                    '页面大小(KB)': len(page.content()) / 1024,
                    '链接数量': len(page.locator("a").all()),
                    '采集时间': time.strftime('%Y-%m-%d %H:%M:%S')
                }
                
                all_data.append(page_data)
                print(f"  ✓ 采集成功: {page_data['页面标题']}")
                
            except Exception as e:
                print(f"  ✗ 采集失败: {e}")
                error_data = {
                    '序号': i,
                    '页面标题': '采集失败',
                    'URL': url,
                    '状态码': '错误',
                    '页面大小(KB)': 0,
                    '链接数量': 0,
                    '采集时间': time.strftime('%Y-%m-%d %H:%M:%S')
                }
                all_data.append(error_data)
        
        browser.close()
    
    return pd.DataFrame(all_data)

# 示例：采集多个页面
# 注意：这里使用示例URL，实际使用时请遵守网站服务条款
test_urls = [
    "https://www.example.com",
    # 可以添加更多URL
]

print("开始批量采集...")
df = collect_multiple_pages(test_urls)

# 显示表格
print("\n采集结果表格：")
print(df.to_string(index=False))

# 保存为Excel文件（需要安装openpyxl: pip install openpyxl）
try:
    df.to_excel("批量采集结果.xlsx", index=False, engine='openpyxl')
    print("\n数据已保存到 '批量采集结果.xlsx'")
except ImportError:
    print("\n提示：安装openpyxl可以保存为Excel格式: pip install openpyxl")
    df.to_csv("批量采集结果.csv", index=False, encoding='utf-8-sig')
    print("数据已保存为CSV格式")

# 显示统计摘要
print("\n=== 统计摘要 ===")
print(f"总页面数: {len(df)}")
print(f"成功采集: {len(df[df['状态码'] == '200'])}")
print(f"失败数量: {len(df[df['状态码'] == '错误'])}")
if '页面大小(KB)' in df.columns:
    print(f"平均页面大小: {df['页面大小(KB)'].mean():.2f} KB")
if '链接数量' in df.columns:
    print(f"总链接数: {df['链接数量'].sum()}")


## 总结

### 关键知识点回顾

1. **Selenium vs Playwright**
   - Selenium：成熟稳定，社区庞大
   - Playwright：现代化，性能更好，API更简洁

2. **核心概念**
   - 元素定位：ID、类名、CSS选择器、XPath
   - 等待机制：显式等待优于隐式等待
   - 浏览器控制：真实浏览器环境

3. **HTTP基础**
   - 请求/响应结构
   - GET vs POST
   - HTTPS加密

4. **安全与合规**
   - 遵守robots.txt
   - 遵守服务条款
   - 添加适当延迟
   - 仅用于合法目的

### 下一步学习建议

1. **深入学习**
   - 学习XPath和CSS选择器高级用法
   - 了解Page Object Model设计模式
   - 学习处理动态内容和AJAX请求

2. **实践项目**
   - 构建自己的测试框架
   - 实现数据采集工具
   - 自动化重复性任务

3. **相关技术**
   - 学习HTTP协议深入知识
   - 了解Web安全最佳实践
   - 学习数据库存储采集的数据

### 资源推荐

- **官方文档**
  - Selenium: https://www.selenium.dev/documentation/
  - Playwright: https://playwright.dev/python/

- **学习网站**
  - MDN Web Docs（Web技术文档）
  - OWASP（Web安全知识）

---

**记住**：能力越大，责任越大。请始终以合法、道德的方式使用这些工具！
